In [1]:
import scipy
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt 
import scipy
import math
import matplotlib.patches as patches
import pylab
import shapely.geometry as geometry
import pylab as plt
from matplotlib.path import Path
import scipy.stats
import shapefile
from matplotlib.path import Path

Reading in ensemble, interpolated to 5km grid using nearest neighbor. 

In [2]:
ensemble = xr.open_dataset('double_ens/indus_interp.nc')

Reading in synthetic observations. Frequency does not account for "wet" snow (aka wet snow is included)

In [3]:
obs = xr.open_dataset('obs_wetsnow_v2.nc')

Reading in uninterpolated ensemble, just for data cleaning.

In [4]:
check = xr.open_dataset('../files_for_pbs/ensemble_final1.nc')
ensemble['time'] = check.time
del check
ensemble_snow = ensemble.snd
ensemble_snow = ensemble_snow.load()
del ensemble

In [7]:
lon = ensemble_snow.lon
lat = ensemble_snow.lat

In [5]:
ensemble_snow = ensemble_snow.transpose('ensemble', 'time', 'lat', 'lon')
ensemble_snow = ensemble_snow.sel(time=slice('2016-10', '2017-09'))
ensemble_snow['ensemble'] = np.arange(100)

# Assimilating all observations

Initializing weights. Initially, all weights are the same (1). I normalize the weights later, so it doesn't matter that their not 1/n. 

In [8]:
weights = np.ones([340, 300, 100])
weights = xr.DataArray(weights, coords = {'lon': lon, 'lat': lat, 'ensemble': np.arange(100)}, dims = ['lon', 'lat', 'ensemble'])
weights = weights.transpose('ensemble', 'lat', 'lon')
#weights = weights.where(mask)

Selecting and cleaning snow depth observations

In [9]:
obs_SD = obs.SnowDepth_tavg
del obs
obs_SD = obs_SD.rename({'east_west': 'lon', 'north_south': 'lat'})

In [10]:
obs_SD = obs_SD.load()

Assimilaiton algorithm for assimilating all observations

In [13]:
for j in np.arange(100):
    #Loop through each ensemble member 
    mem = ensemble_snow.sel({'ensemble': j})
    #Observation uncertainty has the same standard deviation 
    #as the uncertainty that I specified when creating the 
    #synthetic observations 
    std = (obs_SD + 0.01)*0.30  
    #Taking the difference between the ensemble member and 
    #the syntehic observation 
    dif = mem - obs_SD 
    del mem 
    dif = dif.load()
    #Calculating the Likelihood. Std is descirbed above. 
    #Bias is set to -0.01, because this matches the bias of the
    #Sentinel-1 data
    lik = scipy.stats.norm.pdf(dif, -0.01, std)
    del dif
    #Replacing nans with 1 (When calculating joint likelihood, 
    #the one will have no affect)
    lik = np.nan_to_num(lik, nan = 1)
    #Replacing zero likelihoods with very small numbers to 
    #prevent div by zeros 
    zeros = lik == 0
    lik[zeros] = 3.8 * 10**(-322)
    #Calculating the joint log-likelihood. 
    #Using log-likelihood prevents the joint likelihoods from
    #collapsing to zero. When using normal joint likelihood
    #the numbers become so small, that python turns them
    #into zeros. 
    joint_lik = np.log(lik[0])
    for i in np.arange(1, 273):
        elem = np.log(lik[i])
        joint_lik = joint_lik + elem
    #Updating weights 
    weights[j] = joint_lik
    del joint_lik 
    del lik

Normalizing weights 

In [14]:
#First, sum up the weights for each ensemble, to caclulate 
#the normalizing factor 
log_add = weights.sel({'ensemble':0})
for i in np.arange(1, 100):
    add = weights.sel({'ensemble': i})
    #np.logaddexp adds logs together but somehow doesn't 
    #round the small numbers to zero, like python does. 
    log_add = np.logaddexp(log_add, add)
#Now we can normalize the weights, still using log 
normalized_log = weights - log_add
#Finally, we exponentiate the weights 
normalized_all = np.exp(normalized_log)
normalized_all = normalized_all.load()

Save the weights as a netcdf

In [15]:
normalized_all.to_netcdf('all_assim_weights_wetsnow_v2.nc')

## Assimilating maximum

In [16]:
del weights
del normalized_all

In [17]:
del log_add

Initialize weights

In [18]:
weights = np.ones([340, 300, 100])
weights = xr.DataArray(weights, coords = {'lon': lon, 'lat': lat, 'ensemble': np.arange(100)}, dims = ['lon', 'lat', 'ensemble'])
weights = weights.transpose('ensemble', 'lat', 'lon')
#weights = weights.where(mask)

Filling zeros with very small numbers (they won't be the maximum, so this nans are still ignored)

In [19]:
obs_SD_fill = obs_SD.fillna(0.0000001)

Getting the index of the OBSERVATION maximum value

In [20]:
obs_max_ind = obs_SD_fill.argmax('time', skipna = True) 

Getting the OBSERVATION maximum value and calculating the observation 
uncertainty of this maximum value

In [21]:
obs_max = obs_SD_fill.max('time', skipna= True)
#obs_max = obs_max.where(mask)
std = (obs_max + 0.01) * 0.15

In [22]:
for i in np.arange(100):
    #Looping through each ensemble member
    mem = ensemble_snow.sel({'ensemble': i})
    #Getting the model SD value, for where the observation
    #maximum is. 
    max_mem = mem[obs_max_ind]
    max_mem = max_mem.load()
    #Calculating the error and likelihood 
    error = max_mem - obs_max 
    lik = scipy.stats.norm.pdf(error, -0.01, std)
    #Replacing zeros with very small numbers to avoid 
    #div by zero.
    zeros = lik == 0
    lik[zeros] = 3.8 * 10**(-322)
    weights[i] = np.log(lik)
#Normalizing weights, as before
log_add = weights.sel({'ensemble':0})
for i in np.arange(1, 100):
    add = weights.sel({'ensemble': i})
    log_add = np.logaddexp(log_add, add)
normalized_log = weights - log_add
normalized_max = np.exp(normalized_log)

Saving weights as netcdf

In [23]:
normalized_max.to_netcdf('max_assim_weights_wetsnow_v2.nc')